In [1]:
import sys
sys.path.insert(0, "/home/pwahle/proliferating_RGC/")

import yaml
import importlib
import modules
importlib.reload(modules)
import os
from pathlib import Path
import subprocess

# load global variables and parameters
with open("../params.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

globals().update(cfg)

from scipy import ndimage
import re
import cv2
from scipy import ndimage as ndi
import copy
import numpy as np
import matplotlib as mpl
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

In [2]:
def affine_matrix_from_points(v0, v1, shear=True, scale=True, usesvd=True):

    v0 = np.array(v0, dtype=np.float64, copy=True)
    v1 = np.array(v1, dtype=np.float64, copy=True)

    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")

    # move centroids to origin
    t0 = -np.mean(v0, axis=1)
    M0 = np.identity(ndims+1)
    M0[:ndims, ndims] = t0
    v0 += t0.reshape(ndims, 1)
    t1 = -np.mean(v1, axis=1)
    M1 = np.identity(ndims+1)
    M1[:ndims, ndims] = t1
    v1 += t1.reshape(ndims, 1)

    if shear:
        # Affine transformation
        A = np.concatenate((v0, v1), axis=0)
        u, s, vh = np.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]
        C = vh[ndims:2*ndims]
        t = np.dot(C, np.linalg.pinv(B))
        t = np.concatenate((t, np.zeros((ndims, 1))), axis=1)
        M = np.vstack((t, ((0.0,)*ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        # Rigid transformation via SVD of covariance matrix
        u, s, vh = np.linalg.svd(np.dot(v1, v0.T))
        # rotation matrix from SVD orthonormal bases
        R = np.dot(u, vh)
        if np.linalg.det(R) < 0.0:
            # R does not constitute right handed system
            R -= np.outer(u[:, ndims-1], vh[ndims-1, :]*2.0)
            s[-1] *= -1.0
        # homogeneous transformation matrix
        M = np.identity(ndims+1)
        M[:ndims, :ndims] = R
    else:
        # Rigid transformation matrix via quaternion
        # compute symmetric matrix N
        xx, yy, zz = np.sum(v0 * v1, axis=1)
        xy, yz, zx = np.sum(v0 * np.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = np.sum(v0 * np.roll(v1, -2, axis=0), axis=1)
        N = [[xx+yy+zz, 0.0,      0.0,      0.0],
             [yz-zy,    xx-yy-zz, 0.0,      0.0],
             [zx-xz,    xy+yx,    yy-xx-zz, 0.0],
             [xy-yx,    zx+xz,    yz+zy,    zz-xx-yy]]
        # quaternion: eigenvector corresponding to most positive eigenvalue
        w, V = np.linalg.eigh(N)
        q = V[:, np.argmax(w)]
        q /= vector_norm(q)  # unit quaternion
        # homogeneous transformation matrix
        M = quaternion_matrix(q)

    if scale and not shear:
        # Affine transformation; scale is ratio of RMS deviations from centroid
        v0 *= v0
        v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))

    # move centroids back
    M = np.dot(np.linalg.inv(M1), np.dot(M, M0))
    M /= M[ndims, ndims]
    return M

In [3]:
path = data_path + 'exp5/'

In [4]:
from bs4 import BeautifulSoup
from bs4 import BeautifulStoneSoup 
with open(path + 'multipoints.xml', 'rb') as f:
    data = f.read()
Bs_orig = BeautifulSoup(data, "xml")

with open(path + 'multipoints_after.xml', 'rb') as f:
    data = f.read()
Bs_new = BeautifulSoup(data, "xml")

In [5]:
a1 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[0]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[0]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[0]['value'])])
a2 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[1]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[1]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[1]['value'])])
a3 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[2]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[2]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[2]['value'])])
v0 = np.array([a1,a2,a3])

b1 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[0]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[0]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[0]['value'])])
b2 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[1]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[1]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[1]['value'])])
b3 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[2]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[2]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[2]['value'])])
v1 = np.array([b1,b2,b3])

T = affine_matrix_from_points(v0, v1,shear=False, scale=False, usesvd=True)

In [6]:
bX = []
for i in Bs_new.find_all('dXPosition', {'runtype':'double'})[3:]:
    bX.append(float(i['value']))
bY = []
for i in Bs_new.find_all('dYPosition', {'runtype':'double'})[3:]:
    bY.append(float(i['value']))
bZ = []
for i in Bs_new.find_all('dZPosition', {'runtype':'double'})[3:]:
    bZ.append(float(i['value']))
v3 = np.squeeze(np.dstack([bX,bY,bZ,np.ones(len(bZ))]))

In [16]:
np.set_printoptions(suppress=True)
projected_points = np.matmul(v3,T)
projected_points

array([[  14505.96253426,    8514.94530386,    5157.8030887 ,
        1396113.23124491],
       [  14921.19579192,    8118.84318304,    5172.91414491,
        1321467.57062025],
       [  14117.27345789,    8561.07686342,    5161.89106107,
        1404210.38468295],
       [  13674.77929897,    8661.54898015,    5122.57950186,
        1424303.93448062],
       [  14432.83852816,    8596.01707071,    5133.80003158,
        1412290.19470988],
       [  26368.92371568,    4264.69260077,    5253.61069242,
         605546.27955037],
       [  26322.21671017,    4614.9648645 ,    5225.11200824,
         672525.2380048 ],
       [  26299.38674353,    4987.13005036,    5243.46969476,
         741655.26357155],
       [  26251.54790162,    5083.35535   ,    5242.20754826,
         759741.10189153],
       [  38000.24006344,    -929.66937679,    5386.51688083,
        -364234.60124076],
       [  38012.56497974,    -795.44168745,    5380.36656758,
        -338739.0092625 ],
       [  38047.04788

In [ ]:
for tag in Bs_new.find_all('dXPosition', {'runtype':'double'}):
    tag['value'] = str(float(tag['value'])/100)

In [ ]:
with open(path + "test.xml", "w") as f:
    f.write(Bs_data.prettify())

In [1]:
import sys
sys.path.insert(0, "/home/pwahle/proliferating_RGC/")

import yaml
import importlib
import modules
import utils
importlib.reload(modules)
import os
from pathlib import Path
import subprocess

# load global variables and parameters
with open("../params.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

globals().update(cfg)

from scipy import ndimage
import re
import cv2
from scipy import ndimage as ndi
import copy
import numpy as np
import matplotlib as mpl
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from bs4 import BeautifulSoup




updated points in file multipoints_cycle2_updated.xml


In [15]:
v0

array([[14263.4 , 14431.3 , 38106.  , 38882.5 ],
       [ 9783.9 ,  8495.6 ,  -905.3 ,   229.4 ],
       [ 5337.6 ,  5335.58,  5285.27,  5285.27]])

In [11]:
v1

array([[14345.5 ,  9491.3 ,  5345.7 ],
       [14520.2 ,  8205.  ,  5343.68],
       [38236.9 , -1091.1 ,  5277.55],
       [39010.  ,    48.8 ,  5272.6 ]])

In [13]:
a4

array([38882.5 ,   229.4 ,  5285.27])

In [18]:
v3.T

array([[2.64324e+04, 2.64240e+04, 2.64230e+04, 2.64190e+04, 2.64093e+04],
       [4.45100e+03, 4.50530e+03, 4.56450e+03, 4.61630e+03, 4.68890e+03],
       [5.30377e+03, 5.30367e+03, 5.30281e+03, 5.30377e+03, 5.30169e+03],
       [1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00]])